In [7]:
import logging
from google.auth import credentials
from google.auth.transport.requests import Request
from google.cloud import bigquery
import json
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import math
import statsmodels.api as sm
from typing import Union
import logging
import time

# Define your Google Cloud project ID (this can also come from your GitHub secrets)
project_id = "logistics-data-storage-staging"  # Replace with your actual project ID or use a secret for the project_id
logging.info(f"Initializing BigQuery client for project: {project_id}")

# Load the credentials from the file created in the GitHub Action workflow
credentials_path = '/tmp/credentials.json'

# Check if the credentials file exists
if os.path.exists(credentials_path):
    # Load credentials from the file
    with open(credentials_path, 'r') as f:
        creds_data = json.load(f)
    
    # Rebuild credentials object using the loaded data
    credentials = credentials.Credentials.from_authorized_user_info(creds_data)
    
    # Refresh credentials if necessary
    if credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
else:
    raise FileNotFoundError(f"Credentials file not found at {credentials_path}")

# Now initialize the BigQuery client with the credentials and project_id
try:
    client = bigquery.Client(credentials=credentials, project=project_id)
    logging.info(f"BigQuery client initialized successfully for project: {project_id}")
except Exception as e:
    logging.error(f"Failed to initialize BigQuery client: {e}")
    raise e

# Define the query
test_query = """
Select *
FROM `logistics-data-storage-staging.long_term_pricing.abc_performance` 
"""

# Run the query and store the result in a DataFrame
test_df = client.query(test_query).to_dataframe()

# Display the result
print(test_df)

FileNotFoundError: Credentials file not found at /tmp/credentials.json